## Installing Ollama dependencies
---

1. `pciutils` is required by Ollama to detect the GPU type.
2. Installation of Ollama in the runtime instance will be taken care by `curl -fsSL https://ollama.com/install.sh | sh`

In [13]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
35 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

## Running Ollama
---

In order to use Ollama it needs to run as a service in background parallel to your scripts. Because Jupyter Notebooks is built to run code blocks in sequence this make it difficult to run two blocks at the same time. As a workaround we will create a service using subprocess in Python so it doesn't block any cell from running.

Service can be started by command `ollama serve`.

`time.sleep(5)` adds some delay to get the Ollama service up before downloading the model.

In [14]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

## Runing project

In [15]:
%pip install dotenv weave langchain_core langchain_openai langchain_ollama

In [16]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key_preview = os.getenv("OPENAI_API_KEY")[:10]
print(f"First 10 characters of API key: {api_key_preview}")

wandb_key_preview = os.getenv("WANDB_API_KEY")[:10]
print(f"First 10 characters of W&B key: {wandb_key_preview}")

First 10 characters of API key: sk-proj-WL
First 10 characters of W&B key: 2cc6e41e14


In [17]:
import weave
from langchain_core.prompts import PromptTemplate

In [18]:
weave.init("langchain_demo")

In [19]:
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI()
# prompt = PromptTemplate.from_template("1 + {number} = ")

# llm_chain = prompt | llm

# output = llm_chain.invoke({"number": 2})

# print(output)

In [ ]:
model = 'llama3'
!ollama pull $model

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏  11 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  33 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  47 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   2% ▕▏  86 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   2% ▕▏ 115 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   3% ▕▏ 132 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   4% ▕▏ 164 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   4% ▕▏ 189 MB/4.7 GB  189 MB/s   

In [ ]:
from langchain_ollama.chat_models import ChatOllama

# Initialize the ChatOllama model
model_llama = ChatOllama(
    model=model,  # Specify the model version
    base_url="http://localhost:11434",  # URL where Ollama is running locally
    temperature=0.7,  # Control the randomness of the output (0.0 to 1.0)
)

# Note: Ensure Ollama is running on your computer before executing this code

# If you encounter an OllamaEndpointNotFoundError, you may need to pull the model
# Run the following command in your terminal:
# ollama pull llama3.1

# Generate a response from the model
response = model_llama.invoke("Olá, meu nome é Yuri. Qual é o seu nome?")

# Print the response
print(response)